## Synthesising single samples from a trained model

In [1]:
import tensorflow as tf
import numpy as np
import json
from IPython.display import display, Audio
from tqdm import tqdm
import librosa

#### Get the trained model and class labels

In [4]:
path_to_generator = 'C:/nmb/nada/Jongho/checkpoints/18-04-2021_17h/generator.h5'
path_to_labels = 'C:/nmb/nada/Jongho/checkpoints/18-04-2021_17h/label_names.json'
z_dim = 100
sample_rate = 22050

In [5]:
#load the generator
generator = tf.keras.models.load_model(path_to_generator)

In [6]:
#read the labels from the generated dictionary during training
with open(path_to_labels) as json_file:
    label_names = json.load(json_file)
label_names

{'0': 'F', '1': 'M'}

#### Generating a single sample (with label)

In [7]:
#create noise and label
label = 0
noise = np.random.normal(0,1, (1, z_dim))
label_synth = np.array(label).reshape(-1,1)

#synthesise the audio
%time synth_audio = generator.predict([noise, label_synth])

#listen to the synthesised audio
# display(Audio(np.squeeze(synth_audio[0]), rate = sample_rate))

import sounddevice as sd

sd.play(np.squeeze(synth_audio[0]), sample_rate)

ValueError: in user code:

    C:\Users\ai\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1478 predict_function  *
        return step_function(self, iterator)
    C:\Users\ai\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\ai\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\ai\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\ai\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\ai\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1461 run_step  **
        output

### Batch generation

In [61]:
#how many samples per label
n_samples_label = 1200

In [62]:
for emotion in tqdm(label_names):
    noise  = tf.random.normal(shape=(n_samples_label, z_dim))
    label_synth = tf.constant(int(emotion), shape=(n_samples_label,1))
    synth_audio = generator.predict([noise, label_synth])
    for i in range(n_samples_label):
        librosa.output.write_wav(f'C:/nmb/nmb_data/new_data/{label_names[emotion]}_{i}.wav', y = np.squeeze(synth_audio[i]), sr = sample_rate, norm=False) 

100%|██████████| 2/2 [00:18<00:00,  9.08s/it]
